In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

RS = 56
data = pd.read_csv('./bankDataset/bank-full.csv', sep=';')

for column in data.select_dtypes(include=['object']).columns:
    data[column] = LabelEncoder().fit_transform(data[column])

X = data.drop('y', axis=1)
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RS)

smote = SMOTE(random_state=RS)
X_res, y_res = smote.fit_resample(X_train, y_train)

rf = RandomForestClassifier(random_state=RS)
rf.fit(X_res, y_res)

y_pred = rf.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.880128276014597
